# 02 — Feature Engineering

**Objective:** Prepare data for modeling through missing value handling, encoding, scaling, and feature creation. No data leakage.

## 1. Load Raw Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

df = pd.read_csv("../data/raw/churn-bigml-20_raw.csv")
df.head()

## 2. Missing Value Handling

**Business rationale:** [Describe strategy—e.g., drop rows with >X% missing, impute numeric with median, categorical with mode.]

In [ ]:
# Example: if no missing, skip; else impute
df_clean = df.copy()
if df_clean.isnull().any().any():
    df_clean = df_clean.fillna(df_clean.median(numeric_only=True))
    df_clean = df_clean.fillna(df_clean.mode().iloc[0])
df_clean.to_csv("../data/processed/churn_clean.csv", index=False)
print("Cleaned data shape:", df_clean.shape)

## 3. Encoding Explanation

**Categorical features:** `State`, `International plan`, `Voice mail plan`

**Strategy:** One-hot encode binary (Yes/No), label encode or one-hot for State. Avoid leakage by fitting on train only.

In [ ]:
df_feat = df_clean.copy()
df_feat["International plan"] = (df_feat["International plan"] == "Yes").astype(int)
df_feat["Voice mail plan"] = (df_feat["Voice mail plan"] == "Yes").astype(int)
df_feat = pd.get_dummies(df_feat, columns=["State"], drop_first=True)
df_feat["Churn"] = df_feat["Churn"].astype(int)
df_feat.head()

## 4. Scaling Explanation

**Rationale:** StandardScaler for numeric features so models (e.g., Logistic Regression) are not biased by feature scale. Fit on train, transform train and test.

In [ ]:
target = "Churn"
X = df_feat.drop(columns=[target])
y = df_feat[target]

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
X_scaled[target] = y
X_scaled.to_csv("../data/processed/churn_features.csv", index=False)
print("Feature-engineered data shape:", X_scaled.shape)

## 5. Feature Creation Logic (Optional)

**Business-derived features:** e.g., total minutes, total charge, calls per minute ratio. Justified by domain knowledge.

In [ ]:
# Example: total usage minutes
# df_feat["total_minutes"] = df_feat["Total day minutes"] + df_feat["Total eve minutes"] + df_feat["Total night minutes"]
# Ensure no leakage: only use past/current info, not future

## 6. Summary

- **No leakage:** Train/test split happens in modeling; scaler/encoder fit on train only.
- **Outputs:** `churn_clean.csv`, `churn_features.csv` in `data/processed/`